In [29]:
from typing import List, Optional
import pandas as pd; pd.set_option('display.max_columns', 100)
from pandas.core.common import flatten
import numpy as np
import shap
from datetime import datetime

from tqdm.notebook import tqdm

import re
import os

from functools import partial
from scipy.stats import mode
from scipy.spatial.distance import cdist

import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns
# import plotly.express as px

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler,PowerTransformer, FunctionTransformer, OneHotEncoder
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV, RepeatedStratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, RocCurveDisplay, cohen_kappa_score, log_loss, f1_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import RFE, RFECV
from sklearn.isotonic import IsotonicRegression
from sklearn.calibration import CalibrationDisplay
from sklearn.inspection import PartialDependenceDisplay, permutation_importance
from sklearn.linear_model import LogisticRegression
from collections import Counter
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

from sklearn.manifold import TSNE
# import optuna
import catboost as cb
import xgboost as xgb

from View import MyClass
mc = MyClass()

import warnings
warnings.filterwarnings('ignore')

In [30]:
params_cb = {
        "n_estimators": 1993,
        "learning_rate": 0.009906548674016816,
        "subsample": 0.5000411100811685,
#         'colsample_bylevel': 0.5853177112754828,
        "loss_function": "Logloss",
        "eval_metric": "AUC",
        "task_type": "CPU",
        #"max_bin": 20,
        "verbose": False,
        "max_depth": 5,
        "l2_leaf_reg": 20,
        "early_stopping_rounds": 100,
#         "thread_count": 6,
        # "random_seed": 42,
        "class_weights" : (1, 2),
        "random_state": 42
        }
params_lgbm = {"objective":'binary',
          "n_estimators" : 3000,
                             "max_depth" : 6,
                             "learning_rate" : 0.01,
                             "num_leaves" : 20,
                             "reg_alpha" : 4,
                             "reg_lambda" : 4,
                             "subsample" : 0.7,
                             "colsample_bytree" : 0.7,
                             "verbose" : -100,
                              "min_data_in_bin": 1,
                              "min_data_in_leaf" : 1
}
params_xgb = {
            # "objective":'binary',
            'max_delta_step': 3,
            'min_child_weight': 2,
            "n_estimators" : 2466,
            'alpha': 2,
             'colsample_bytree': 0.7,
             'eta': 0.003419782719339618,
             'gamma': 1,
             'lambda': 1,
             'max_depth': 8,
             'subsample': 0.31098721316506717
}

In [31]:
print(mc.__doc__)

Визуализация и предобработка данных
_________________________________________________
* Метод "view_csv" принимает DataFrame и выводит данные о нем
возвращает categorical, numerical

* Метод "stat_frame" принимает DataFrame и возвращает DataFrame со статистиками

* Метод corr(data, target):
data: DataFrame
target : str

Работа с данными
__________________________________________________

* Графики
* Метод connect_sign(train, test, columns) Определяет похожи ли признаки в train и test
train, test: DataFrame
columns: list( список колонок без целевой переменной)

* Метод distribution_sign(train, numerical, target_name='target', log=False): распределение признаков
train: DataFrame
numerical: признаки числовые
target_name: str имя целевой переменной
log: bool если нужно логарифмировать признаки

* Метод similarity_sign(data, data_test, numericaal): похожесть признаков и вычисление p-value
data, data_test: DataFrame
numericaal: list числовые признаки

* Метод transformer(train, test, cont_co

In [32]:
os.listdir('data')

['sample_submission.csv', 'test.csv', 'train.csv']

In [33]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
sub = pd.read_csv('data/sample_submission.csv')

In [34]:
cat, num = mc.view_csv(train)
cat_t, num_t = mc.view_csv(test)

rows: 165034, cols: 14

Категориальные признаки: 
Всего категориальных признаков: 3
surname уникальных значений: 2797
geography уникальных значений: 3, ['France' 'Spain' 'Germany']
gender уникальных значений: 2, ['Male' 'Female']

Числовые признаки: 

Всего числовых признаков: 11
['exited', 'creditscore', 'balance', 'tenure', 'numofproducts', 'isactivemember', 'estimatedsalary', 'age', 'id', 'customerid', 'hascrcard']
 для проверки isna: 0
 для проверки isna: 0
 для проверки isna: 0
 для проверки isna: 0
 для проверки isna: 0
 для проверки isna: 0
 для проверки isna: 0
 для проверки isna: 0
 для проверки isna: 0
 для проверки isna: 0
 для проверки isna: 0
 для проверки isna: 0
 для проверки isna: 0
 для проверки isna: 0
В данных пропусков нету
rows: 110023, cols: 13

Категориальные признаки: 
Всего категориальных признаков: 3
surname уникальных значений: 2708
geography уникальных значений: 3, ['France' 'Germany' 'Spain']
gender уникальных значений: 2, ['Female' 'Male']

Числовые призна

In [35]:
train['exited'].value_counts()

exited
0    130113
1     34921
Name: count, dtype: int64

In [36]:
130113//34921

3

In [37]:
train = train.drop("surname", axis=1)
test = test.drop("surname", axis=1)

In [38]:
aggs = ['geography', 'gender']

In [39]:
train, test = mc.encoder(train, test, aggs)

тип кодирования 1: OneHot, 2: LabelEncoder, 3: frequency 1


Готово, размер:  (165034, 18) (110023, 17)


In [40]:
train = train.drop(aggs, axis=1)
test = test.drop(aggs, axis=1)

In [41]:
model_cb, dict_fold_cb=  mc.fit_kfold(train, target='exited', model=cb.CatBoostClassifier ,name_model="catboost",params=params_cb, trh_flag=True, threshold=0.5, fold=10, eval_set=False )

Fold 0 ==> catboost model ROC-AUC score is ==> 0.8916054094396608
Fold 1 ==> catboost model ROC-AUC score is ==> 0.8876294778033774
Fold 2 ==> catboost model ROC-AUC score is ==> 0.8907635792355211
Fold 3 ==> catboost model ROC-AUC score is ==> 0.8878453680421
Fold 4 ==> catboost model ROC-AUC score is ==> 0.8866894106608003
Fold 5 ==> catboost model ROC-AUC score is ==> 0.8948137812370061
Fold 6 ==> catboost model ROC-AUC score is ==> 0.8878077700224227
Fold 7 ==> catboost model ROC-AUC score is ==> 0.8937882810060356
Fold 8 ==> catboost model ROC-AUC score is ==> 0.8886974260831195
Fold 9 ==> catboost model ROC-AUC score is ==> 0.8859956853849015
Признаки для удаления set()
Лучших 5  признаков: {'1', 'balance', 'numofproducts', 'isactivemember', 'age'}
Средняя совокупность оценок ROC-AUC за 10-кратный период составляет 0.8895636188914946


In [42]:
trh = 0.5
pred = model_cb.predict_proba(test)[:,1]
# pred = np.where(pred> trh, 1, 0)
sub["Exited"] = pred
if os.path.exists("submission") == False:
    os.mkdir("submission")
sub.to_csv("submission/submissions.csv", index=False)

In [22]:
sub.head()

,id,Exited
0,165034,0.5
1,165035,0.5
2,165036,0.5
3,165037,0.5
4,165038,0.5
